In [1]:
import pandas as pd
import numpy as np
import os
import requests
import math
import env
import wrangle as w
import explore as exp
import datetime
from scipy.stats import chi2_contingency
# Importing necessary libraries for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import chi2

In [2]:
# Example usage:
app_token = env.app_token
year_to_retrieve = '2022'
max_req = 2000  # Specify the maximum number of observations to retrieve

In [3]:
df = w.wrangle_coll_stage2(year_to_retrieve, app_token)

CSV file for 2022 already exists. Loading data from the CSV.


In [4]:
df.shape

(48254, 24)

In [5]:
df.to_csv('features_df.csv', index=False)

In [6]:
df.head()

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,on_street_name,borough,zip_code,injuries_count,...,vehicle_2_category,factors_category_vehicle_1,factors_category_vehicle_2,factors_subcat_vehicle_1,factors_subcat_vehicle_2,hour_of_day,day_of_week,month,daylight,daylight_day_of_week
0,2022-01-01 05:17:00,2022-01-01,05:17:00,4491857,40.746930,-73.84866,grand central pkwy,queens,11368,1,...,personal vehicles,driver_related,driver_related,driving violations,driving violations,5,Saturday,January,False,Saturday_Night
1,2022-01-01 01:30:00,2022-01-01,01:30:00,4491344,40.819157,-73.96038,henry hudson parkway,manhattan,10027,0,...,personal vehicles,non_driver_related,non_driver_related,environmental,environmental,1,Saturday,January,False,Saturday_Night
2,2022-01-01 16:40:00,2022-01-01,16:40:00,4491478,40.806107,-73.91799,saint ann's avenue,bronx,10454,0,...,personal vehicles,non_driver_related,non_driver_related,environmental,environmental,16,Saturday,January,True,Saturday_Day
3,2022-01-01 02:53:00,2022-01-01,02:53:00,4491586,40.646034,-73.99678,40th street,brooklyn,11232,0,...,personal vehicles,non_driver_related,non_driver_related,environmental,environmental,2,Saturday,January,False,Saturday_Night
4,2022-01-01 17:00:00,2022-01-01,17:00:00,4491660,40.701195,-73.91409,wyckoff avenue,brooklyn,11237,0,...,personal vehicles,driver_related,non_driver_related,driving violations,environmental,17,Saturday,January,True,Saturday_Day


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48254 entries, 0 to 54203
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   crash_datetime              48254 non-null  datetime64[ns]
 1   crash_date                  48254 non-null  datetime64[ns]
 2   crash_time                  48254 non-null  object        
 3   collision_id                48254 non-null  int64         
 4   latitude                    48254 non-null  float64       
 5   longitude                   48254 non-null  float64       
 6   on_street_name              48254 non-null  object        
 7   borough                     48253 non-null  object        
 8   zip_code                    48254 non-null  int64         
 9   injuries_count              48254 non-null  int64         
 10  deaths_count                48254 non-null  int64         
 11  injuries                    48254 non-null  bool          


In [8]:
from sklearn.model_selection import train_test_split
# Splitting the data into train, validation, and test sets with a 70-15-15 ratio
train, temp = train_test_split(df, test_size=0.3, random_state=42, stratify=df['injuries'])
val, test = train_test_split(temp, test_size=0.5, random_state=42, stratify=temp['injuries'])

# Checking the shape of the datasets to confirm the split ratio
train.shape, val.shape, test.shape

((33777, 24), (7238, 24), (7239, 24))

In [9]:
train.collision_id.nunique(),val.collision_id.nunique(),test.collision_id.nunique()

(33777, 7238, 7239)

In [10]:
from sklearn.cluster import DBSCAN

# Initialize DBSCAN
dbscan = DBSCAN(eps=0.002, min_samples=15, metric='manhattan')

# Prepare the data from the train set and fit the model
injury_data_train = train[train['injuries']][['latitude', 'longitude']].values
dbscan_labels_train = dbscan.fit_predict(injury_data_train)

# Prepare the data from the validation and test sets
injury_data_val = val[val['injuries']][['latitude', 'longitude']].values
dbscan_labels_val = dbscan.fit_predict(injury_data_val)
injury_data_test = test[test['injuries']][['latitude', 'longitude']].values
dbscan_labels_test = dbscan.fit_predict(injury_data_test)

# Create new DataFrames to hold the cluster labels and unique identifiers
train_clusters = train[train['injuries']][['collision_id']].copy()
train_clusters['cluster'] = dbscan_labels_train
val_clusters = val[val['injuries']][['collision_id']].copy()
val_clusters['cluster'] = dbscan_labels_val
test_clusters = test[test['injuries']][['collision_id']].copy()
test_clusters['cluster'] = dbscan_labels_test

# Merge these new DataFrames back into the original train, validation, and test sets
train = train.merge(train_clusters, on='collision_id', how='left')
val = val.merge(val_clusters, on='collision_id', how='left')
test = test.merge(test_clusters, on='collision_id', how='left')

# Fill NaN cluster labels with a new cluster label that indicates 'noise' or 'unclassified'
noise_label = -1
train['cluster'].fillna(noise_label, inplace=True)
val['cluster'].fillna(noise_label, inplace=True)
test['cluster'].fillna(noise_label, inplace=True)

# Convert cluster labels to integers
train['cluster'] = train['cluster'].astype(int)
val['cluster'] = val['cluster'].astype(int)
test['cluster'] = test['cluster'].astype(int)

In [11]:

# Group by cluster label and count the number of injuries in each cluster
train_clusters['cluster_injury_count'] = train_clusters.groupby('cluster')['collision_id'].transform('count')

# Remove duplicate cluster labels to have a unique set
unique_train_clusters = train_clusters.drop_duplicates(subset=['cluster']).copy()

# Sort by cluster_injury_count
unique_train_clusters = unique_train_clusters.sort_values('cluster_injury_count')

# Create a new column cluster_new_name based on the order of the sort
unique_train_clusters['cluster_new_name'] = range(len(unique_train_clusters))
# Ensure that cluster -1 should remain -1
unique_train_clusters.loc[unique_train_clusters['cluster'] == -1, 'cluster_new_name'] = -1

# Map the 'cluster_new_name' and 'cluster_injury_count' from 'unique_train_clusters' to the original train, validation, and test sets
mapping_new_name = dict(unique_train_clusters[['cluster', 'cluster_new_name']].values)
mapping_injury_count = dict(unique_train_clusters[['cluster', 'cluster_injury_count']].values)

train['cluster_new_name'] = train['cluster'].map(mapping_new_name)
train['cluster_injury_count'] = train['cluster'].map(mapping_injury_count)

val['cluster_new_name'] = val['cluster'].map(mapping_new_name)
val['cluster_injury_count'] = val['cluster'].map(mapping_injury_count)

test['cluster_new_name'] = test['cluster'].map(mapping_new_name)
test['cluster_injury_count'] = test['cluster'].map(mapping_injury_count)

# Fill NaN values for 'cluster_new_name' and 'cluster_injury_count' with -1 and 0 respectively
train['cluster_new_name'].fillna(-1, inplace=True)
train['cluster_injury_count'].fillna(0, inplace=True)

val['cluster_new_name'].fillna(-1, inplace=True)
val['cluster_injury_count'].fillna(0, inplace=True)

test['cluster_new_name'].fillna(-1, inplace=True)
test['cluster_injury_count'].fillna(0, inplace=True)

# Convert the new columns to integer type
train['cluster_new_name'] = train['cluster_new_name'].astype(int)
train['cluster_injury_count'] = train['cluster_injury_count'].astype(int)

val['cluster_new_name'] = val['cluster_new_name'].astype(int)
val['cluster_injury_count'] = val['cluster_injury_count'].astype(int)

test['cluster_new_name'] = test['cluster_new_name'].astype(int)
test['cluster_injury_count'] = test['cluster_injury_count'].astype(int)

# Drop the original 'cluster' column and rename 'cluster_new_name' to 'cluster'
train.drop(columns=['cluster'], inplace=True)
val.drop(columns=['cluster'], inplace=True)
test.drop(columns=['cluster'], inplace=True)

train.rename(columns={'cluster_new_name': 'cluster'}, inplace=True)
val.rename(columns={'cluster_new_name': 'cluster'}, inplace=True)
test.rename(columns={'cluster_new_name': 'cluster'}, inplace=True)

# Verify if the new features have been correctly added without changing the number of rows
train.shape, val.shape, test.shape


((33777, 26), (7238, 26), (7239, 26))

In [12]:
# # Convert 'crash_datetime' to datetime object for train, val, and test sets
# train['crash_datetime'] = pd.to_datetime(train['crash_datetime'])
# val['crash_datetime'] = pd.to_datetime(val['crash_datetime'])
# test['crash_datetime'] = pd.to_datetime(test['crash_datetime'])

# Create a reference date (New Year 2022)
ref_date = pd.Timestamp('2022-01-01 00:00:00')

# Calculate the time since the reference date in days
train['time_since_ref_date'] = (train['crash_datetime'] - ref_date).dt.days
val['time_since_ref_date'] = (val['crash_datetime'] - ref_date).dt.days
test['time_since_ref_date'] = (test['crash_datetime'] - ref_date).dt.days

# Drop 'crash_datetime', 'crash_date', 'crash_time'
train.drop(['crash_datetime', 'crash_date', 'crash_time'], axis=1, inplace=True)
val.drop(['crash_datetime', 'crash_date', 'crash_time'], axis=1, inplace=True)
test.drop(['crash_datetime', 'crash_date', 'crash_time'], axis=1, inplace=True)



In [13]:
# Drop 'crash_datetime', 'crash_date', 'crash_time'
train.drop(['injuries_count', 'deaths_count', 'deaths', 'collision_id'], axis=1, inplace=True)
val.drop(['injuries_count', 'deaths_count', 'deaths', 'collision_id'], axis=1, inplace=True)
test.drop(['injuries_count', 'deaths_count', 'deaths', 'collision_id'], axis=1, inplace=True)

In [14]:
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import MinMaxScaler

# Select only the numerical columns
numerical_columns = train.select_dtypes(include=['int64', 'float64', 'int32'])
# numerical_columns = train.drop(columns=['collision_id'], axis=1)

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the scaler on the training data
train[numerical_columns.columns] = scaler.fit_transform(train[numerical_columns.columns])

# Transform the validation and test data using the same scaler
val[numerical_columns.columns] = scaler.transform(val[numerical_columns.columns])
test[numerical_columns.columns] = scaler.transform(test[numerical_columns.columns])

In [15]:
train.to_csv('train.csv', index=False)

In [16]:
val.to_csv('val.csv', index=False)

In [17]:
test.to_csv('test.csv', index=False)

In [18]:
encode_cols = train.drop(columns=['on_street_name']).select_dtypes(include=['object']).columns
train_encoded = pd.get_dummies(train, columns=encode_cols)
val_encoded = pd.get_dummies(val, columns=encode_cols)
test_encoded = pd.get_dummies(test, columns=encode_cols)

In [19]:
# Extract features and target variable from the training set
# Here, we include both numerical and properly encoded categorical variables
X_train = train_encoded.drop(['injuries', 'on_street_name'], axis=1)
y_train = train_encoded['injuries']

# Initialize SelectKBest with chi2
selector_kbest = SelectKBest(score_func=chi2, k=5)

# Fit SelectKBest
selector_kbest = selector_kbest.fit(X_train, y_train)

# Get the scores
kbest_scores = pd.Series(selector_kbest.scores_, index=X_train.columns)
kbest_scores

latitude                                <scipy.stats._distn_infrastructure.rv_continuo...
longitude                               <scipy.stats._distn_infrastructure.rv_continuo...
zip_code                                <scipy.stats._distn_infrastructure.rv_continuo...
hour_of_day                             <scipy.stats._distn_infrastructure.rv_continuo...
daylight                                <scipy.stats._distn_infrastructure.rv_continuo...
                                                              ...                        
daylight_day_of_week_Thursday_Night     <scipy.stats._distn_infrastructure.rv_continuo...
daylight_day_of_week_Tuesday_Day        <scipy.stats._distn_infrastructure.rv_continuo...
daylight_day_of_week_Tuesday_Night      <scipy.stats._distn_infrastructure.rv_continuo...
daylight_day_of_week_Wednesday_Day      <scipy.stats._distn_infrastructure.rv_continuo...
daylight_day_of_week_Wednesday_Night    <scipy.stats._distn_infrastructure.rv_continuo...
Length: 80

In [20]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2

# Extract features and target variable from the training set
X_train = train.drop(['injuries'], axis=1).select_dtypes(include=['int64', 'float64', 'int32'])
y_train = train['injuries']

# Initialize a Logistic Regression model
model = LogisticRegression()

# Initialize RFE with Logistic Regression
selector_rfe = RFE(estimator=model, n_features_to_select=5)

# Fit RFE
selector_rfe = selector_rfe.fit(X_train, y_train)

# Get the ranking of features
rfe_ranking = pd.Series(selector_rfe.ranking_, index=X_train.columns)

# Initialize and fit Logistic Regression with L1 penalty
lasso = LogisticRegression(penalty='l1', solver='liblinear').fit(X_train, y_train)

# Get feature importance
lasso_importance = pd.Series(np.abs(lasso.coef_[0]), index=X_train.columns)

# Combine all feature rankings
feature_ranking = pd.DataFrame({
    'RFE': rfe_ranking,
    # 'SelectKBest': kbest_scores,
    'Lasso': lasso_importance
})

# Display the combined feature rankings
feature_ranking


,RFE,Lasso
latitude,1,0.321981
longitude,1,0.564198
zip_code,1,0.656290
hour_of_day,2,0.293727
cluster,1,2.645642
cluster_injury_count,1,4.465980
time_since_ref_date,3,0.270305


In [21]:
# Calculate the correlation matrix for the encoded training data
train_corr = train.drop(['injuries'], axis=1).select_dtypes(include=['int64', 'int32', 'float64'])
# Calculate the correlation matrix for the encoded training data
correlation_matrix = train_corr.corr()
correlation_matrix

,latitude,longitude,zip_code,hour_of_day,cluster,cluster_injury_count,time_since_ref_date
latitude,1.000000,0.319235,-0.500001,-0.017709,0.036061,-0.046876,-0.003438
longitude,0.319235,1.000000,0.460853,-0.005029,0.015767,-0.009018,-0.007987
zip_code,-0.500001,0.460853,1.000000,0.001821,-0.004246,0.015795,-0.002840
hour_of_day,-0.017709,-0.005029,0.001821,1.000000,-0.015437,0.016362,0.005601
cluster,0.036061,0.015767,-0.004246,-0.015437,1.000000,-0.893854,0.001799
cluster_injury_count,-0.046876,-0.009018,0.015795,0.016362,-0.893854,1.000000,-0.000624
time_since_ref_date,-0.003438,-0.007987,-0.002840,0.005601,0.001799,-0.000624,1.000000


___
___

## Modeling

___
___

baseline

In [22]:
# Calculate the baseline accuracy for the 'injuries' column
most_frequent_class = train['injuries'].value_counts().idxmax()
baseline_accuracy = (train['injuries'] == most_frequent_class).mean()
# baseline_accuracy = round(baseline_accuracy * 100, 1)
baseline_accuracy


0.6709595286733576

In [23]:
# Temporarily concatenate the train, val, and test datasets
temp_df = pd.concat([train, val, test], keys=['train', 'val', 'test'])

# One-hot encode the categorical columns
encode_cols = temp_df.drop(columns=['on_street_name']).select_dtypes(include=['object']).columns
temp_df_encoded = pd.get_dummies(temp_df, columns=encode_cols)

# Split the data back into train, val, and test sets
train_encoded = temp_df_encoded.loc['train']
val_encoded = temp_df_encoded.loc['val']
test_encoded = temp_df_encoded.loc['test']

In [24]:
# Importing necessary libraries for Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Defining the features and the target variable
X_train = train_encoded.drop(['injuries'], axis=1).select_dtypes(include=['int64', 'float64', 'int32'])
y_train = train_encoded['injuries']
X_val = val_encoded.drop(['injuries'], axis=1).select_dtypes(include=['int64', 'float64', 'int32'])
y_val = val_encoded['injuries']

# Initializing the Logistic Regression model
logistic_model = LogisticRegression(random_state = 42, max_iter = 500, multi_class= 'multinomial') 

# Fitting the model
logistic_model.fit(X_train, y_train)

# Making predictions on the validation set
y_val_pred = logistic_model.predict(X_val)

# Calculating the accuracy of the model
logistic_accuracy = accuracy_score(y_val, y_val_pred)

# Displaying the accuracy
logistic_accuracy


0.6709035645205857

In [33]:
# Defining the features and the target variable
X_train = train_encoded.drop(['injuries', 'on_street_name'], axis=1)
y_train = train_encoded['injuries']
X_val = val_encoded.drop(['injuries', 'on_street_name'], axis=1)
y_val = val_encoded['injuries']

In [34]:
from sklearn.ensemble import RandomForestClassifier

# Initializing the Random Forest Classifier
rf_model = RandomForestClassifier(random_state=42, n_estimators=100)

# Fitting the model to the training data
rf_model.fit(X_train, y_train)

# Making predictions on the validation set
y_val_pred_rf = rf_model.predict(X_val)

# Calculating the accuracy of the model on the validation set
rf_accuracy = accuracy_score(y_val, y_val_pred_rf)

# Rounding the accuracy to keep it concise
rf_accuracy = round(rf_accuracy * 100, 1)

rf_accuracy

68.5

In [35]:
from sklearn.ensemble import GradientBoostingClassifier

# Initializing the Gradient Boosting Classifier
gb_model = GradientBoostingClassifier(random_state=42, n_estimators=100)

# Fitting the model to the training data
gb_model.fit(X_train, y_train)

# Making predictions on the validation set
y_val_pred_gb = gb_model.predict(X_val)

# Calculating the accuracy of the model on the validation set
gb_accuracy = accuracy_score(y_val, y_val_pred_gb)

# Rounding the accuracy to keep it concise
gb_accuracy = round(gb_accuracy * 100, 1)

gb_accuracy


70.5

In [36]:
from sklearn.svm import SVC

# Initializing the Support Vector Machine (SVM) model
# Using a linear kernel for simplicity and faster computation
svm_model = SVC(kernel='linear', random_state=42)

# Fitting the model to the training data
svm_model.fit(X_train, y_train)

# Making predictions on the validation set
y_val_pred_svm = svm_model.predict(X_val)

# Calculating the accuracy of the model on the validation set
svm_accuracy = accuracy_score(y_val, y_val_pred_svm)

# Rounding the accuracy to keep it concise
svm_accuracy = round(svm_accuracy * 100, 1)

svm_accuracy


69.9

In [39]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='accuracy')

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best parameters from the grid search
best_params = grid_search.best_params_

# Use the best parameters to initialize a new Random Forest Classifier
best_rf = RandomForestClassifier(**best_params, random_state=42)
best_rf.fit(X_train, y_train)

# Calculate train and validation accuracies
rf_train_accuracy = round(accuracy_score(y_train, best_rf.predict(X_train)) * 100, 1)
rf_val_accuracy = round(accuracy_score(y_val, best_rf.predict(X_val)) * 100, 1)

# Add these accuracies to the results DataFrame
new_row = pd.DataFrame({'Model': ['Random Forest (Grid Search)'], 'Train Accuracy (%)': [rf_train_accuracy], 'Validation Accuracy (%)': [rf_val_accuracy]})
results_df = pd.concat([results_df, new_row], ignore_index=True)

NameError: name 'param_grid' is not defined

In [25]:
X_train

,latitude,longitude,zip_code,hour_of_day,cluster,cluster_injury_count,time_since_ref_date
0,0.454685,0.607739,0.898138,0.608696,0.000000,1.000000,0.708791
1,0.545631,0.643323,0.931736,0.739130,0.000000,1.000000,0.406593
2,0.365898,0.562323,0.894286,0.956522,0.000000,1.000000,0.958791
3,0.611606,0.583720,0.872459,0.913043,0.000000,1.000000,0.645604
4,0.566074,0.622016,0.931522,0.565217,0.000000,1.000000,0.914835
...,...,...,...,...,...,...,...
33772,0.621443,0.689896,0.930452,0.826087,0.000000,1.000000,0.799451
33773,0.384102,0.658432,0.895142,0.043478,0.000000,1.000000,0.766484
33774,0.622490,0.683046,0.930452,0.130435,0.000000,1.000000,0.593407
33775,0.544284,0.958415,0.942007,0.608696,0.000000,1.000000,0.107143
